In [16]:
import gdal
import os
import datetime

In [2]:
fn='/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf'

In [17]:
YEAR=2018

In [23]:
import re

In [33]:
tilepattern='.h\d\dv\d\d.'

In [34]:
print(re.search(tilepattern,fn).group())

.h16v02.


In [37]:
datepatern='.A\d\d\d\d\d\d\d.'
print(re.search(datepatern,fn).group())

.A2018175.


In [22]:
fn.split('MCD43A3.A')[1].split('.')[0].split(str(YEAR))[1]

'175'

# Get the list of subdataset
hdf gather the different bands as sub datasets

In [3]:
sds=gdal.Open(fn).GetSubDatasets()

## Print the list of subdatasets

In [4]:
for i,s in enumerate(sds):
    print(i,s[0])

0 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band1
1 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band2
2 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band3
3 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band4
4 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band5
5 HDF4_EOS:EOS_GRID:"/home/gbessardon/DATA/MCD43A3/061/2018/MCD43A3.A2018175.h16v02.061.2021335155343.hdf":MOD_Grid_BRDF:BRDF_Albedo_Band_Mandatory_Quality_Band6
6 HDF4_EOS:EOS_GRID:"/home/g

# Open the different bands

In [5]:
dsBSANIR=gdal.Open(sds[18][0])
dsNIRQC=gdal.Open(sds[8][0] )
dsBSAVIS=gdal.Open(sds[17][0])
dsVISQC=gdal.Open(sds[7][0])

In [6]:
geoT = dsBSAVIS.GetGeoTransform()
projT = dsBSAVIS.GetProjection()
w=dsBSAVIS.RasterXSize
h=dsBSAVIS.RasterYSize

In [7]:
def create_raster(output_path,columns,rows,nband = 1,gdal_data_type = gdal.GDT_Int16, driver = r'GTiff'):
    ''' returns gdal data source raster object

    '''
    # create driver
    driver = gdal.GetDriverByName(driver)

    output_raster = driver.Create(output_path,
                                  int(columns),
                                  int(rows),
                                  nband,
                                  eType = gdal_data_type)    
    return output_raster

In [8]:

def numpy_array_to_raster(output_path,
                          numpy_array,
                          geoTin,
                          projectionin,
                          nband = 1,
                          no_data = -9999,
                          gdal_data_type = gdal.GDT_Int16,
                          driver = r'GTiff'):
    ''' returns a gdal raster data source

    keyword arguments:

    output_path -- full path to the raster to be written to disk
    numpy_array -- numpy array containing data to write to raster
    geoTin -- geotransform input from ds.GetGetGeoTransform()
    projectionin -- projection input from ds.GetProjection()
    nband -- the band to write to in the output raster
    no_data -- value in numpy array that should be treated as no data
    gdal_data_type -- gdal data type of raster (see gdal documentation for list of values)
    driver -- string value of the gdal driver to use

    '''


    rows, columns = numpy_array.shape

    # create output raster
    output_raster = create_raster(output_path,
                                  int(columns),
                                  int(rows),
                                  nband,
                                  gdal_data_type) 

    geotransform = geoTin


    output_raster.SetProjection(projectionin)
    output_raster.SetGeoTransform(geotransform)
    output_band = output_raster.GetRasterBand(1)
    output_band.SetNoDataValue(no_data)
    output_band.WriteArray(numpy_array)          
    output_band.FlushCache()
    output_band.ComputeStatistics(False)

    if os.path.exists(output_path) == False:
        raise Exception('Failed to create raster: %s' % output_path)

    return  

In [9]:
for item,value in dsBSANIR.GetMetadata_Dict().items():
    if item=='add_offset':
        offset=int(value)
    if item=='scale_factor':
        scale_factor=float(value)
    if item=='_FillValue':
        fillvalue=int(value)

In [10]:
numpy_array_to_raster(os.path.join(os.getcwd(),'test.tif'),
                      dsBSANIR.ReadAsArray(),
                      geoT,
                      projT,
                      nband = 1,
                      no_data = fillvalue,
                      gdal_data_type = gdal.GDT_Int16,
                      driver = r'GTiff')

In [ ]:
gdal.Translate('test.tif',NIRQC,format='Gtiff')

In [ ]:
ds=gdal.Open('test.tif')

In [ ]:
ds.ReadAsArray()

In [ ]:
vi_np=BSANIR.ReadAsArray()
QA_np =NIRQC.ReadAsArray()

In [ ]:
# Clean data
vi_np[np.logical_and(QA_np != 0, QA_np != 1)] = -3000
# De-allocate QA array
QA_np = None

In [ ]:
help(BSAVIS.GetProjection)